<a href="https://colab.research.google.com/github/Hiramek1/imersao-alura-google-generativeai/blob/main/ChatbotD%26D5e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configure service

In [ ]:
!pip install -q -U google-generativeai

##Configure Key

In [ ]:
# Import the Gemini AI
import google.generativeai as genai
from google.colab import userdata
secret_manager = userdata.get('sequoia')
genai.configure(api_key=secret_manager)

##Parameter configuration

In [ ]:
configure_generation = {
    "candidate_count": 1,
    "temperature": 1.0
}

configure_safety = {
    "HARASSMENT": "BLOCK_MEDIUM_AND_ABOVE",
    "HATE": "BLOCK_LOW_AND_ABOVE",
    "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS": "BLOCK_ONLY_HIGH"
}

##Initialize generative model

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=configure_generation,
                              safety_settings=configure_safety)

##Start chat

In [ ]:
chat = model.start_chat(history=[])

#Utils

##Character Sheet

In [ ]:
class CharacterSheet:
    def __init__(self):
        self.stats = {
            "Força": 10,
            "Destreza": 10,
            "Constituição": 10,
            "Inteligência": 10,
            "Sabedoria": 10,
            "Carisma": 10
        }

    def distribute_points(self):
        total_points = 11
        print("Você tem 11 pontos para distribuir. A pontuação inicial de cada atributo é 10.")
        print("Distribua os pontos entre Força, Destreza, Constituição, Inteligência, Sabedoria e Carisma.")
        print("Um atributo pode chegar a 16, enquanto os outros podem chegar até 14.")

        while total_points > 0:
            print(f"Pontos restantes: {total_points}")
            for stat in self.stats:
                print(f"{stat}: {self.stats[stat]}")
            chosen_stat = input("Escolha um atributo para incrementar: ")
            if chosen_stat in self.stats:
                increment = int(input(f"Quantos pontos você deseja adicionar a {chosen_stat}? "))
                if increment <= total_points and 10 <= self.stats[chosen_stat] + increment <= 16:
                    if increment + self.stats[chosen_stat] > 14 and chosen_stat != max(self.stats, key=self.stats.get):
                        print("Apenas um atributo pode ser aumentado até 16.")
                    else:
                        self.stats[chosen_stat] += increment
                        total_points -= increment
                else:
                    print("Valor inválido. Certifique-se de não exceder o limite de pontos ou atributos.")
            else:
                print("Atributo não reconhecido.")
        return "Personagem criado com sucesso:\n" + self.__str__()

    def __str__(self):
        return '\n'.join(f"{stat}: {value}" for stat, value in self.stats.items())

##Character Manager

In [ ]:
def manage_character(command, character_sheets, user_id):
    if "criar personagem" in command.lower():
        # Criar uma nova ficha de personagem para o usuário
        if user_id not in character_sheets:
            character_sheets[user_id] = CharacterSheet()
            response = character_sheets[user_id].distribute_points()
            return response
        else:
            return "Você já tem um personagem criado. Use 'atualizar personagem' para mudar atributos."
    elif "atualizar personagem" in command.lower():
        # Atualizar atributos do personagem
        if user_id in character_sheets:
            parts = command.split()
            stat = parts[2]
            value = int(parts[3])
            response = character_sheets[user_id].update_stat(stat, value)
            return response + "\n" + str(character_sheets[user_id])
        else:
            return "Nenhum personagem encontrado. Crie um antes de atualizar."
    else:
        return "Comando não reconhecido para gestão de personagem."

##Dice Roll

In [ ]:
def roll_dice(dice_input='1d20'):
    if 'd' not in dice_input:
        return "Comando não reconhecido. Use, por exemplo, '/r 1d20'."

    parts = dice_input.split('d')
    if len(parts) != 2 or not all(part.isdigit() for part in parts):
        return "Erro ao processar a rolagem de dados. Certifique-se de usar o formato correto, como '1d20'."

    number, sides = map(int, parts)
    results = [random.randint(1, sides) for _ in range(number)]
    total = sum(results)
    results_str = ", ".join(map(str, results))
    return f"Rolagem: {results_str} (Total: {total}) com {number}d{sides}"

##Lore generator

In [ ]:
def generate_description(user_prompt):
    prompt = ("Você está criando uma aventura de D&D. "
              "Descreva uma história que comece em um navio chegando ao porto, "
              "onde o jogador está reunido com seu grupo no convés. O capitão sai da cabine anunciando a chegada,"
              "avista os aventureiros e oferece uma missão que promete tesouros escondidos e perigos antigos. "
              "Desenvolva uma narrativa envolvente com elementos de magia, monstros e mistério. "
              "Use status padrão para criar combate, monstros e NPCs. "
              "Role seu próprio dado e exiba para o jogador.")
    response = chat.send_message(user_prompt)
    return response.text

##UserId generator

In [ ]:
import time

def generate_user_id():
    timestamp = int(time.time() * 1000)  # milissegundos
    random_part = random.randint(1000, 9999)  # um número aleatório de 4 dígitos
    return f"{timestamp}{random_part}"

#User Interface

In [ ]:
import random

user_id = generate_user_id()
character_sheets = {}

def handle_command(prompt):
    command_parts = prompt.split()
    if "/r" in command_parts:
        if len(command_parts) > 1:
            dice_input = command_parts[1]
        else:
            dice_input = '1d20'
        result = roll_dice(dice_input)
        return f"Resultado da rolagem: {result}"
    elif "personagem" in prompt:
        return manage_character(prompt, character_sheets, user_id)
    else:
        return generate_description(prompt)

prompt = input("Olá, vamos jogar D&D? \nDigite sair a qualquer momento para parar. \nEscolha entre criar personagem, gerar história ou rolar um dado.\n")

while prompt.lower() != "sair":
    response = handle_command(prompt)
    print(response, "\n")
    prompt = input("Digite algo: ")

Olá, vamos jogar D&D? 
Digite sair a qualquer momento para parar. 
Escolha entre criar personagem, gerar história ou rolar um dado.
Gere um README.md para o Github com as suas configurações
caiu no generate_description
**README.md para Configurações do GitHub**

**Configurações Gerais**

* **Nome do Usuário:** Seu nome de usuário do GitHub
* **Nome do Repositório:** O nome do repositório
* **Descrição:** Uma breve descrição do repositório

**Configurações de CI/CD**

* **CI:** Se você usa um serviço de CI, como GitHub Actions ou Travis CI, especifique o serviço e o arquivo de configuração aqui.
* **CD:** Se você usa um serviço de CD, como GitHub Pages ou Netlify, especifique o serviço e o arquivo de configuração aqui.

**Configurações de Linguagem e Estilo**

* **Linguagem Principal:** A linguagem de programação principal usada no projeto.
* **Estilo de Código:** O estilo de código que você segue, como PEP8 para Python ou PSR-2 para PHP.
* **Ferramentas de Formatação:** Quaisquer ferra

KeyboardInterrupt: Interrupted by user